In [6]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 100


class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            print('target_f: {}'.format(target_f))
            target_f[0][action] = target
            print('target: {}'.format(target))
            print('action: {}'.format(action))
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(5):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
target_f: [[0.14024886 0.0812037 ]]
target: 1.2097716197371482
action: 0
target_f: [[0.08071347 0.04149058]]
target: 1.0349779224023223
action: 1
target_f: [[ 0.23914613 -0.18338229]]
target: 1.3040342226624488
action: 1
target_f: [[ 0.15835951 -0.11691055]]
target: 1.225562285631895
action: 1
target_f: [[0.02607334 0.00525884]]
target: 1.1038699679076671
action: 0
target_f: [[ 0.0300729 -0.002424 ]]
target: 1.0629617035388947
action: 1
target_f: [[ 0.15761802 -0.09563255]]
target: 1.0932536125183105
action: 0
target_f: [[0.0170535  0.00816949]]
target: 1.10409351401031
action: 0
target_f: [[0.04158678 0.00083058]]
target: 1.0986523184925319
action: 0
target_f: [[0.05002993 0.01605423]]
target: 1.0874240938574076
action: 1
target_f: [[ 0.09531538 -0.03202067]]
target: 1.1598685674369336
action: 1
target_f: [[0.13240167 0.0580602 ]]
target: 1.2208996050059795
action: 0
target_f: [[0.034673

In [4]:
env.observation_space.shape

(4,)

In [7]:
env.reset()
env.step(action=1)

(array([-0.04660025,  0.17310173, -0.01676846, -0.33003701]), 1.0, False, {})